# 2020 1C 3ra Oportunidad

2) Mercadodesregulado tiene un dataframe de ventas (id_producto, id_comprador, id_vendedor, pais, tiene_reclamo, año, mes, precio_local)
y uno de productos (id_producto, titulo, categoria), quiere conocer:

a) Los peores vendedores, aquellos cuya tasa de reclamos sea mayor a 0.3, para retirarlos de la plataforma (10 ptos)

b) El smartphone cuya cantidad de ventas se redujera más en diciembre de 2019 respecto de diciembre de 2018 (10 ptos)

c) Se observa que la mayoría de los peores vendedores retirados de la plataforma con la consulta de (a) son producto de la llamada "ecuación más peligrosa de la historia" o “ecuación de Demoivre”.
Proponga una mejora a la consulta para mitigar esto. (5 ptos)


In [80]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random as rd
%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [131]:
ventas = pd.DataFrame({
 'id_producto': rd.choices(range(1,6),k=200),
 'id_comprador': rd.choices(range(1,16),k=200),
 'id_vendedor': rd.choices(range(1,10),k=200),
 'pais': rd.choices(['Argentina','Brasil','Chile','Paraguay','Perú'],k=200),
 'tiene_reclamo': rd.choices([0,1],weights=[90,10],k=200),
 'año': pd.date_range(start='2018-01-01',freq='W',periods=200).year,
 'mes': pd.date_range(start='2018-01-01',freq='W',periods=200).month,
 'precio_local':rd.choices(range(1,10000),k=200)
})
ventas

,id_producto,id_comprador,id_vendedor,pais,tiene_reclamo,año,mes,precio_local
0,2,5,4,Paraguay,0,2018,1,4197
1,2,7,6,Argentina,0,2018,1,5145
2,3,6,7,Perú,1,2018,1,5025
3,4,1,8,Perú,1,2018,1,1117
4,2,10,4,Perú,0,2018,2,1077
...,...,...,...,...,...,...,...,...
195,5,6,3,Paraguay,0,2021,10,6858
196,1,5,3,Brasil,0,2021,10,1951
197,5,11,9,Chile,0,2021,10,682
198,2,1,4,Argentina,0,2021,10,6116


In [132]:
len(ventas[ventas['tiene_reclamo']==1].index)

22

In [133]:
productos = pd.DataFrame({
 'id_producto': range(1,6),
 'titulo': ['Celular1','Celular2','Celular3','Zapatilla','Perfume'],
 'categoria': ['Smartphone']*3 + ['Vestimenta','Hogar']
})
productos

,id_producto,titulo,categoria
0,1,Celular1,Smartphone
1,2,Celular2,Smartphone
2,3,Celular3,Smartphone
3,4,Zapatilla,Vestimenta
4,5,Perfume,Hogar


## Ejercicio a)

In [134]:
#obtengo cant_reclamos
vendedores_reclamos = ventas.groupby('id_vendedor').sum().reset_index()[['id_vendedor','tiene_reclamo']]
vendedores_reclamos

,id_vendedor,tiene_reclamo
0,1,2
1,2,3
2,3,4
3,4,0
4,5,0
5,6,3
6,7,3
7,8,5
8,9,2


In [135]:
#ventas totales por cenedor
ventas['cantidad_ventas'] = 1
vendedores_ventas_totales = ventas.groupby('id_vendedor').count().reset_index()[['id_vendedor','cantidad_ventas']]
vendedores_ventas_totales

,id_vendedor,cantidad_ventas
0,1,15
1,2,23
2,3,26
3,4,19
4,5,26
5,6,26
6,7,30
7,8,18
8,9,17


In [136]:
vendedores_tasa_reclamos = pd.merge(vendedores_reclamos,vendedores_ventas_totales,on='id_vendedor') 
vendedores_tasa_reclamos

,id_vendedor,tiene_reclamo,cantidad_ventas
0,1,2,15
1,2,3,23
2,3,4,26
3,4,0,19
4,5,0,26
5,6,3,26
6,7,3,30
7,8,5,18
8,9,2,17


In [137]:
vendedores_tasa_reclamos['tasa_reclamos'] = vendedores_tasa_reclamos['tiene_reclamo']/vendedores_tasa_reclamos['cantidad_ventas']
vendedores_tasa_reclamos.loc[vendedores_tasa_reclamos['tasa_reclamos']>0.1,['id_vendedor','tasa_reclamos']]

,id_vendedor,tasa_reclamos
0,1,0.13
1,2,0.13
2,3,0.15
5,6,0.12
7,8,0.28
8,9,0.12


## Ejercicio b

In [167]:
mask_año = (ventas['año']==2018) | (ventas['año']==2019)
mask_dic = ventas['mes']==12
ventas_dic = ventas.loc[mask_año&mask_dic][['id_producto','año']]
ventas_dic 

,id_producto,año
47,4,2018
48,1,2018
49,5,2018
50,5,2018
51,2,2018
99,3,2019
100,2,2019
101,4,2019
102,3,2019
103,2,2019


In [217]:
productos_dic_cat = pd.merge(ventas_dic,productos,on='id_producto',how='left')
productos_dic_cat

,id_producto,año,titulo,categoria
0,4,2018,Zapatilla,Vestimenta
1,1,2018,Celular1,Smartphone
2,5,2018,Perfume,Hogar
3,5,2018,Perfume,Hogar
4,2,2018,Celular2,Smartphone
5,3,2019,Celular3,Smartphone
6,2,2019,Celular2,Smartphone
7,4,2019,Zapatilla,Vestimenta
8,3,2019,Celular3,Smartphone
9,2,2019,Celular2,Smartphone


In [218]:
smartphones_dic_año = productos_dic_cat.loc[productos_dic_cat['categoria']=='Smartphone']
smartphones_dic_año

,id_producto,año,titulo,categoria
1,1,2018,Celular1,Smartphone
4,2,2018,Celular2,Smartphone
5,3,2019,Celular3,Smartphone
6,2,2019,Celular2,Smartphone
8,3,2019,Celular3,Smartphone
9,2,2019,Celular2,Smartphone


In [219]:
smartphones_dic_año = smartphones_dic_año.rename(columns={'categoria':'cant_vendida'})

In [220]:
tipo_celular_por_años = pd.pivot_table(smartphones_dic_año,index='titulo',columns='año',values=['cant_vendida'],aggfunc='count')
tipo_celular_por_años.columns.levels

FrozenList([['cant_vendida'], [2018, 2019]])

In [221]:
tipo_celular_por_años = tipo_celular_por_años['cant_vendida']
tipo_celular_por_años

año,2018,2019
titulo,,
Celular1,1.00,NaN
Celular2,1.00,2.00
Celular3,NaN,2.00


In [222]:
#tipo_celular_por_años['reduccion_anual'] = tipo_celular_por_años['2018']-tipo_celular_por_años['2019']
tipo_celular_por_años = tipo_celular_por_años.reset_index()
tipo_celular_por_años

año,titulo,2018,2019
0,Celular1,1.00,NaN
1,Celular2,1.00,2.00
2,Celular3,NaN,2.00


In [228]:
tipo_celular_por_años['reduccion_venta'] = tipo_celular_por_años[2018] - tipo_celular_por_años[2019]
tipo_celular_por_años.nlargest(1,columns='reduccion_venta')


año,titulo,2018,2019,reduccion_venta
1,Celular2,1.00,2.00,-1.00
